行列分解（個々では特異値分解）

ユーザーとアイテムからなる評価値行列を、ユーザーの因子行列と、アイテムの因子行列に分解する。

In [1]:
import pandas as pd
from src.data import MovieLens20MDataLoader
import os
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from src.recommender.svd import SVDRecommender
from src.data import MovieLens20MDataLoader

In [2]:
ml_loader = MovieLens20MDataLoader()
ml_data = ml_loader.load()

file is found. loading...


In [3]:
print(ml_data.keys())
display(ml_data.get('movies').head())
display(ml_data.get('ratings').head())

dict_keys(['genome-scores', 'genome-tags', 'links', 'movies', 'ratings', 'tags'])


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
# ユーザーごとにチャンク読み込み、結合

_movies = ml_data.get('movies').copy()
_ratings = ml_data.get('ratings').copy()

save_path = f"{MovieLens20MDataLoader().DATA_DIRECTORY}/{MovieLens20MDataLoader().directory_name}/ratings"
os.makedirs(save_path, exist_ok=True)

_size = 20
_loop = len(_ratings.userId.unique().tolist()) // _size + 1

user_limit = 2 # _size * user_limit を上限とする
movie_limit = 40


for index in tqdm(range(_loop)):
    if user_limit * index > user_limit:
        break
    start = index * _size
    end = (index + 1) * _size
    file_name = f"{save_path}/rating_matrix_{start}-{end}.csv"
    _ratings.loc[
        (_ratings["userId"].isin(range(start, end)))
        & (_ratings["movieId"] < movie_limit)
    ].pivot_table(values=["rating"], index=["userId"], columns=["movieId"]).to_csv(file_name)


matrix_data = pd.DataFrame()
for index in tqdm(range(_loop)):
    if user_limit * index > user_limit:
        break
    start = index * _size
    end = (index + 1) * _size
    file_name = f"{save_path}/rating_matrix_{start}-{end}.csv"
    _data = pd.read_csv(file_name, header=1, index_col=0)
    _data = _data.drop("userId") # 謎の行を削除
    matrix_data = pd.concat([matrix_data, _data], axis=0)
    
matrix_data.to_csv(f"{save_path}/rating_matrix_all.csv")

/tmp/ipykernel_1527/1875492515.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm(range(_loop)):


  0%|          | 0/6925 [00:00<?, ?it/s]

/tmp/ipykernel_1527/1875492515.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm(range(_loop)):


  0%|          | 0/6925 [00:00<?, ?it/s]

In [5]:
save_path = f"{MovieLens20MDataLoader().DATA_DIRECTORY}/{MovieLens20MDataLoader().directory_name}/ratings"
matrix_data2 = pd.read_csv(f"{save_path}/rating_matrix_all.csv", index_col=0)
matrix_data2

,1,2,3,5,6,7,10,11,14,15,...,34,36,39,8,9,18,20,22,23,31
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,3.0,NaN,NaN,3.0,NaN,4.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4.0,NaN,5.0,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
save_path = f"{MovieLens20MDataLoader().DATA_DIRECTORY}/{MovieLens20MDataLoader().directory_name}/ratings"
matrix_data2 = pd.read_csv(f"{save_path}/rating_matrix_all.csv", index_col=0)
matrix_data2 = matrix_data2.fillna(matrix_data2.mean())

svd_rec = SVDRecommender(matrix_data2)
svd_rec.fit(k = 5)
svd_rec.recommend(1)

,score
itemId,
18,5.029253
16,4.163847
1,4.140057
23,4.023403
20,4.023403
9,4.023403
25,4.023244
11,4.022316
7,3.832082


In [9]:
svd_rec.P

array([[-2.16817485e-03, -1.55939378e-02,  7.97584851e-03,
         1.45573501e-02, -1.77624882e-01],
       [-1.62885138e-02, -1.22966743e-02, -2.65338599e-02,
         3.79077467e-03, -1.77493480e-01],
       [-8.21148238e-03,  5.01364581e-03, -3.03818861e-03,
        -8.79524548e-02, -1.76836987e-01],
       [-8.96468329e-02, -1.32872594e-01, -1.08844000e-01,
         5.32085600e-01, -1.72159752e-01],
       [ 1.39889544e-01, -1.20539035e-01,  5.59057045e-02,
        -9.13794857e-03, -1.77105941e-01],
       [-1.77728340e-01,  3.74611291e-01,  2.40708053e-02,
         9.48673316e-02, -1.82038488e-01],
       [ 5.08981223e-01, -3.22048188e-01,  9.21868295e-02,
        -6.06749055e-02, -1.66643011e-01],
       [-2.38635367e-01, -1.72750241e-01, -4.97641547e-01,
         5.78744112e-02, -1.76554678e-01],
       [-1.22930782e-02, -1.18295800e-02,  2.83806832e-03,
        -4.88168194e-03, -1.76481642e-01],
       [ 1.20388505e-01,  6.44983655e-02,  1.63783395e-01,
        -3.13212029e-01

In [10]:
svd_rec.S

array([  2.944779  ,   3.65675201,   3.70942451,   4.48330389,
       106.97837916])

In [11]:
svd_rec.Qt

array([[ 2.85224477e-01, -1.50627339e-02, -1.10643714e-01,
         2.28001881e-03,  4.28592239e-01, -2.55402892e-01,
        -2.69106103e-02,  7.22301608e-02,  9.51139227e-02,
        -2.17649040e-01, -1.86709181e-01, -4.35541751e-01,
         1.55486053e-01,  2.71908094e-01, -9.32692783e-02,
        -1.48285511e-01, -6.78140169e-03, -2.63618182e-02,
        -2.49441443e-01,  3.91304897e-01,  1.56939277e-01,
        -6.05929465e-03, -8.07905954e-03, -1.00988244e-02,
        -8.07905954e-03,  1.25518845e-02, -8.07905954e-03,
        -1.82673797e-02],
       [ 2.84531052e-01, -6.74370624e-02, -8.69736686e-02,
        -1.39836486e-02,  2.93734363e-01,  4.00001770e-01,
         9.17710106e-03, -5.55424907e-02,  9.32218281e-02,
         9.06798488e-03,  2.29263415e-02,  4.94088578e-01,
         4.44139314e-02, -2.10706783e-01, -7.59700405e-02,
        -2.29379432e-01, -6.10281916e-02,  9.93976430e-02,
        -4.98829430e-01, -2.52820168e-02, -3.31649934e-02,
        -5.52190518e-02, -7.36